In [1]:
# [START gmail_quickstart]
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import base64
from bs4 import BeautifulSoup
import regex as re
import spacy
import numpy as np
import json
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [2]:
creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
        creds = flow.run_local_server()
        # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
results = service.users().labels().list(userId='me').execute()

In [7]:
user_id = 'me'
threads = service.users().threads().list(userId=user_id).execute().get('threads', [])
results_msg = service.users().messages().list(userId=user_id,labelIds = ['INBOX']).execute()
messages = results_msg.get('messages', [])
threads

[{'id': '16acc54611713413',
  'snippet': 'Hi Shoumik, Are you available at 11 PM IST today? I&#39;m available pretty much anytime today so I just threw out a time. Thanks Paul Mendoza, CEO Mobile 760-917-3753 SigParser pmendoza@sigparser.com',
  'historyId': '12940267'},
 {'id': '16ad59f2cd57694b',
  'snippet': 'Hi Shoumik 👋 Team: whiskart Thanks for installing Workast to Whiskart, glad to have you on board! You mentioned in our sign-up process that you would love to use Workast to capture shared team work and',
  'historyId': '12939776'},
 {'id': '16ad59516bf0246b',
  'snippet': 'Hi Shoumik, Step up this year? 90% off men&#39;s $10 Necklaces, Rings, Bracelets right now ... WOMEN MEN ACCESSORIES HOME DECOR GADGETS Today&#39;s Trending Products Hot Stainless Steel ... ₹ 279 ₹',
  'historyId': '12939644'},
 {'id': '16ad57cf9b169c49',
  'snippet': 'Everything you&#39;ve been eyeing is now on sale pepperfry GET OUR APP pepperfry pepperfry Furniture Decor Lamps &amp; Lighting Mattresses &am

In [82]:
msg = service.users().messages().get(userId='me', id='16ace2b6ea6146b7').execute()
msg

{'id': '16ace2b6ea6146b7',
 'threadId': '16ace2b6ea6146b7',
 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'hi, Trail 1 . https://www.linkedin.com/in/diljeets1994/ https://twitter.com/SethiDiljeet HP.com regards, diljeet singh',
 'historyId': '12941235',
 'internalDate': '1558237101000',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'shoumikgoswami@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a92:aa58:0:0:0:0:0 with SMTP id j85csp2591070ili;        Sat, 18 May 2019 20:38:34 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a5d:4d11:: with SMTP id z17mr14682814wrt.308.1558237114175;        Sat, 18 May 2019 20:38:34 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1558237114; cv=none;        d=google.com; s=arc-20160816;        b=sdlobyj1Kzi0z2UGVmH2OlG9bGUmNGlNEdcbnhBKEef/ZlbIaBv0l50N7+kXiCSEUB         Kh6ThwCj9aLb3oH+fkF0LQWgdW

In [83]:
#define the storage variables
contact_list = []
name_track = []
email_track = []
phone_track = []
twitter_track = []
linkedin_track = []
facebook_track = []
other_contacts = []
contact_name = []
contact_num = []
contact_email = []

In [67]:
#Function to extract names from the string using spacy
def extract_name(string):
    names = []
    nlp = spacy.load('en')
    #nlp = spacy.load('en_core_web_sm')
    doc = nlp(string)
    ent_list = ['PERSON']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            names.append(ent.text)
    return np.unique(names)

In [35]:
def extract_org(string):
    org = []
    nlp = spacy.load('en')
    doc = nlp(string)
    ent_list = ['GPE','NORP','ORG','LOC','PRODUCT']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            org.append(ent.text)
    return np.unique(org)

In [36]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = np.unique(r.findall(string))
    return [re.sub(r'\D', '', number) for number in phone_numbers]

In [37]:
#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return np.unique(r.findall(string))
    #return r.findall(string)

In [38]:
def extract_twitter(string):
    r = re.compile(r'twitter.com/([a-zA-Z0-9])')
    return r.findall(string)

In [39]:
def extract_linkedin(string):
    r = re.compile(r'linkedin.com/([a-zA-Z0-9])')
    return r.findall(string)

In [40]:
def extract_facebook(string):
    r = re.compile(r'facebook.com/([a-zA-Z0-9])')
    return r.findall(string)

In [84]:
df = pd.DataFrame(msg['payload']['headers'])

In [ ]:
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'From']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'From']))))
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'To']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'To']))))
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'Cc']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'Cc']))))

In [ ]:
#get content of the email
#body = msg['payload']['parts'][0]['parts'][0]['body']['data']
body = msg['payload']['parts'][0]['body']['data']
html_data = base64.urlsafe_b64decode(body)
soup = BeautifulSoup(html_data, 'html.parser')

In [ ]:
content = soup.get_text()
string = content.replace(',',' ')
string = string.replace('\n',' ')
string = string.replace('\r',' ')
string = string.replace('\\r\\n',' ')
string1 = string.replace('*','')

In [ ]:
names= extract_name(string1)
org = extract_org(string1).tolist()
emails = extract_email_addresses(string1).tolist()
phone_num = extract_phone_numbers(string1)
twitter = extract_twitter(string1)
linkedin = extract_linkedin(string1)
facebook = extract_facebook(string1)

In [ ]:
for i in range(len(emails)):
    name = []
    start_pos = []
    for match in re.finditer(emails[i], string1):
        start_pos.append(match.start())
    for j in range(len(start_pos)):
        end_pos = start_pos[j] + len(emails[i])
        name = extract_name(string1[start_pos[j]-50:start_pos[j]])
        if len(name) > 0:
            #contact = {'name':str(name),'email':emails[i]}
            #contact_test.append(contact)
            if (len(extract_phone_numbers(string1[start_pos[j]-100:start_pos[j]]))>0):
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append(extract_phone_numbers(string1[start_pos[j]-100:start_pos[j]]))
            elif (len(extract_phone_numbers(string1[end_pos:end_pos + 150]))>0):
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append(extract_phone_numbers(string1[end_pos:end_pos + 150]))
            else:
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append('null')

In [ ]:
contact_list_new = pd.DataFrame()
contact_list_new['contact_name'] = contact_name
contact_list_new['contact_email'] = contact_email
#contact_list_new['contact_num'] = contact_num

In [ ]:
contact_list_new.drop_duplicates(subset='contact_email')

In [76]:
contact_list = contact_list_new.drop_duplicates(subset='contact_email').reset_index()

In [ ]:
#name_track = contact_list['contact_name']
#email_track = contact_list['contact_email']
#for i in names:
#    if i not in name_track:
#        contact = {'name':i,'email':''}
#        other_contacts.append(contact)
#for i in emails:
#    if i not in email_track:
#        contact = {'name':'','email':i}
#        other_contacts.append(contact)

In [77]:
contact_list = contact_list_new.drop_duplicates(subset='contact_email').reset_index().to_dict('index')

In [80]:
response = {'contact': contact_list,
           #'other_contact': other_contacts,
           #'organisation': org,
           'phone number': phone_num,
           'twitter handle': twitter,
           'linkedin handle': linkedin,
           'facebook handle': facebook}

In [81]:
response

{'contact': {0: {'index': 0,
   'contact_name': "['Courtenay Farquharson']",
   'contact_email': "['courtenay.farquharson@g...']"},
  1: {'index': 1,
   'contact_name': '[]',
   'contact_email': "['shoumikgoswami@gmail.com']"},
  2: {'index': 2, 'contact_name': '[]', 'contact_email': '[]'},
  3: {'index': 3,
   'contact_name': "['Courtenay Farquharson']",
   'contact_email': 'courtenay.farquharson@gmail.com'},
  4: {'index': 4,
   'contact_name': "['Courtenay Farquharson']",
   'contact_email': 'courtenay@parserr.com'},
  5: {'index': 5,
   'contact_name': "['Saikumar Jhingade']",
   'contact_email': 'farqy1@hotmail.com'},
  6: {'index': 6,
   'contact_name': "['Elyaas Mohammed']",
   'contact_email': 'v-elmoha@microsoft.com'},
  7: {'index': 7,
   'contact_name': "['Nasmin Kaisar']",
   'contact_email': 'v-naskai@microsoft.com'}},
 'phone number': ['1190504250'],
 'twitter handle': [],
 'linkedin handle': [],
 'facebook handle': []}

In [ ]:
json_resp = json.dumps(response)